In [1]:
print("Hello!")

Hello!


In [18]:
# Load data
import pandas as pd
data_csv = pd.read_csv('./data/new-rumi-sample.csv', index_col=None)
data_csv['Title'].fillna("No Title", inplace=True)
data_csv.head(6)

,Content,Title
0,If anyone asks you\nhow the perfect satisfacti...,No Title
1,Love is the Water of Life\n\nEverything other ...,No Title
2,"A moment of happiness,\n\nyou and I sitting on...",No Title
3,"Lovers\n\nO lovers, lovers it is time\nto set ...",No Title
4,All through eternity\n\nBeauty unveils His exq...,No Title
5,Let go of your worries\n\nand be completely cl...,No Title


In [19]:
len(data_csv)

52

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re

In [21]:
train_test_ratio = 0.9
train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(data_csv, train_size = train_test_ratio, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [22]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Content'].tolist()
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        bos_token = '<BOS>'
        eos_token = '<EOS>'
        data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        
    f.write(data)

In [23]:
build_dataset(df_train, './data/train.txt')
build_dataset(df_valid, './data/valid.txt')
build_dataset(df_test, './data/test.txt')

In [24]:
df_train.head()

,Content,Title
15,Because I cannot sleep\nI make music at night....,No Title
6,"This is love: to fly toward a secret sky,\n\nt...",No Title
47,You come to us\nfrom another world\n\nFrom bey...,THE ALCHEMY OF LOVE\n
7,Love is reckless; not reason.\n\nReason seeks ...,No Title
16,A lifetime without Love is of no account\n\nLo...,No Title


Create Model

In [25]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelWithLMHead.from_pretrained("gpt2")

/Users/willmundy/opt/anaconda3/envs/comp576/lib/python3.7/site-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [26]:
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [27]:
data_csv.apply(lambda x: x["Content"], axis=1)

0     If anyone asks you\nhow the perfect satisfacti...
1     Love is the Water of Life\n\nEverything other ...
2     A moment of happiness,\n\nyou and I sitting on...
3     Lovers\n\nO lovers, lovers it is time\nto set ...
4     All through eternity\n\nBeauty unveils His exq...
5     Let go of your worries\n\nand be completely cl...
6     This is love: to fly toward a secret sky,\n\nt...
7     Love is reckless; not reason.\n\nReason seeks ...
8     I am a sculptor, a molder of form.\n\nIn every...
9     Passion makes the old medicine new:\n\nPassion...
10    The beauty of the heart\n\nis the lasting beau...
11    “I am only the house of your beloved,\n\nnot t...
12    The springtime of Lovers has come,\n\nthat thi...
13    Love has nothing to do with\n\nthe five senses...
14    When the rose is gone and the garden faded\nyo...
15    Because I cannot sleep\nI make music at night....
16    A lifetime without Love is of no account\n\nLo...
17    Last night you lfet me and slept\n\nyour o

100%|██████████| 548118077/548118077 [03:26<00:00, 2649617.02B/s]


Load the models

In [ ]:
# Load data